In [7]:
# !git clone --recursive https://github.com/Microsoft/LightGBM
# ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

In [1]:
!pip install pycaret
from pycaret.regression import *

     |████████████████████████████████| 256kB 6.3MB/s 
     |████████████████████████████████| 6.8MB 16.0MB/s 
     |████████████████████████████████| 65.8MB 105kB/s 
     |████████████████████████████████| 276kB 44.6MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 1.7MB 39.7MB/s 
     |████████████████████████████████| 1.8MB 44.4MB/s 
     |████████████████████████████████| 14.2MB 6.8MB/s 
     |████████████████████████████████| 245kB 29.9MB/s 
     |████████████████████████████████| 174kB 54.0MB/s 
     |████████████████████████████████| 157.5MB 51kB/s 
     |████████████████████████████████| 2.1MB 42.0MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 153kB 48.5MB/s 
     |████████████████████████████████| 348kB 39.6MB/s 
     |████████████████████████████████| 163kB 58.6MB/s 
     |████████████████████████████████| 1.1MB 48.5MB/s

In [2]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt

Mounted at /content/drive


In [17]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

In [8]:
train

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,,,,,
1,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
2,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
3,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
4,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
6,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,A,B,A,C,B,B,A,E,E,L,0.260716,0.712438,0.161661,0.442794,0.768447,0.269578,0.258655,0.363598,0.300619,0.340516,0.235711,0.383477,0.215227,0.793630,8.343538
499996,A,B,A,C,B,B,A,E,E,L,0.173302,0.121591,0.592514,0.193711,0.775951,0.197211,0.257024,0.574304,0.227035,0.322583,0.286094,0.324874,0.306933,0.230902,7.851861
499997,A,B,A,C,B,B,A,E,C,M,0.342856,0.617869,0.462991,0.418098,0.297406,0.449482,0.386172,0.476217,0.135947,0.502730,0.235788,0.316671,0.250286,0.349041,7.600558


In [11]:
reg = setup(train, target = 'target')
b3 = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.7059,7.143000e-01,0.8452,9.450000e-02,0.1025,0.0979,23.294
lightgbm,Light Gradient Boosting Machine,0.7083,7.182000e-01,0.8475,8.950000e-02,0.1028,0.0983,1.877
xgboost,Extreme Gradient Boosting,0.7091,7.232000e-01,0.8504,8.320000e-02,0.1031,0.0983,80.479
gbr,Gradient Boosting Regressor,0.7138,7.289000e-01,0.8537,7.600000e-02,0.1036,0.0991,74.016
rf,Random Forest Regressor,0.7172,7.390000e-01,0.8597,6.310000e-02,0.1042,0.0994,275.626
ridge,Ridge Regression,0.7196,7.476000e-01,0.8646,5.230000e-02,0.1049,0.1000,0.077
br,Bayesian Ridge,0.7197,7.476000e-01,0.8646,5.230000e-02,0.1049,0.1000,0.908
huber,Huber Regressor,0.7174,7.498000e-01,0.8659,4.950000e-02,0.1053,0.1001,7.701
et,Extra Trees Regressor,0.7204,7.511000e-01,0.8667,4.780000e-02,0.1050,0.0999,131.609
omp,Orthogonal Matching Pursuit,0.7278,7.628000e-01,0.8734,3.300000e-02,0.1060,0.1012,0.089


In [13]:
blended = blend_models(estimator_list = b3, fold = 7)
pred_holdout = predict_model(blended)
final_model = finalize_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.7028,0.7085,0.8417,0.0936,0.1021,0.0975
1,0.7132,0.7282,0.8533,0.0930,0.1036,0.0992
2,0.7020,0.7071,0.8409,0.0942,0.1024,0.0972
3,0.7069,0.7144,0.8452,0.0947,0.1022,0.0977
4,0.7058,0.7119,0.8437,0.0968,0.1022,0.0978
5,0.7075,0.7162,0.8463,0.0950,0.1025,0.0981
6,0.7064,0.7166,0.8465,0.0907,0.1028,0.0982
Mean,0.7064,0.7147,0.8454,0.0940,0.1025,0.0980
SD,0.0034,0.0065,0.0038,0.0017,0.0005,0.0006


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.7051,0.7106,0.843,0.0931,0.102,0.0976


In [20]:
train

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,,,,,
1,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
2,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
3,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
4,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
6,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,A,B,A,C,B,B,A,E,E,L,0.260716,0.712438,0.161661,0.442794,0.768447,0.269578,0.258655,0.363598,0.300619,0.340516,0.235711,0.383477,0.215227,0.793630,8.343538
499996,A,B,A,C,B,B,A,E,E,L,0.173302,0.121591,0.592514,0.193711,0.775951,0.197211,0.257024,0.574304,0.227035,0.322583,0.286094,0.324874,0.306933,0.230902,7.851861
499997,A,B,A,C,B,B,A,E,C,M,0.342856,0.617869,0.462991,0.418098,0.297406,0.449482,0.386172,0.476217,0.135947,0.502730,0.235788,0.316671,0.250286,0.349041,7.600558


In [19]:
predictions = predict_model(final_model, data = test)
predictions

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,Label
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,A,B,A,C,B,D,A,E,E,G,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448,7.603337
5,A,B,A,C,B,D,A,E,C,L,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251,7.747756
15,A,B,A,C,B,D,A,E,C,F,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905,7.587223
16,A,A,B,A,B,D,A,E,E,F,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625,7.481483
17,A,B,A,A,B,B,A,E,E,I,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931,7.338538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499987,A,A,A,C,B,D,A,E,G,L,0.919265,0.111147,0.199583,0.181354,0.277365,0.963678,0.240482,0.686462,0.915165,0.848878,0.459598,0.590327,0.864873,0.425258,7.492801
499990,A,A,A,C,B,D,A,E,E,F,0.362875,-0.020197,0.469025,0.336185,0.523174,0.232072,0.363421,0.694092,0.137002,0.319465,0.364527,0.388908,0.664357,0.224215,7.281775
499991,A,A,A,C,B,D,A,E,C,K,0.463060,0.740421,0.446293,0.411387,0.517103,0.432927,0.811876,0.328398,0.496017,0.538779,0.466338,0.643869,0.749590,0.457702,7.523522


In [22]:
submission['target'] = predictions['Label']

In [25]:
submission.to_csv("t.csv", index = True)
!cp t.csv "drive/My Drive/"